# Tracing LangChain with Azure AI Foundry

This notebook explain how to use `langchain-azure-ai` package with the capabilities in Azure AI Foundry for tracing.

## 1. Prerequisites

To run this tutorial you need either:

1. Create an [Azure subscription](https://azure.microsoft.com).
2. Create an Azure AI hub resource as explained at [How to create and manage an Azure AI Studio hub](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/create-azure-ai-resource).
3. Deploy one model supporting the [Azure AI model inference API](https://aka.ms/azureai/modelinference). In this example we use a `Mistral-Large-2407` and a `Mistral-Small` deployment. 

    * You can follow the instructions at [Add and configure models to Azure AI model inference service](https://learn.microsoft.com/azure/ai-studio/ai-services/how-to/create-model-deployments).

4. Go to the section **Tracing** in Azure AI Foundry portal and copy the instrumentation connection string.
5. Configure the following environment variables (or update the code in the notebook directly):

    * `AZURE_APPINSIGHT_CONNECTION`: The instrumentation connection string, pointing to the Azure Application Insights used by the hub or project.
    * `AZURE_INFERENCE_ENDPOINT`: The endpoint where the models are deployed.
    * `AZURE_INFERENCE_CREDENTIAL`: The credentials to connect to the mode.

Install the following packages:

```bash
pip install -U langchain-core langchain-azure-ai[opentelemetry]
```

> Notice that we are installing the extra `langchain-azure-ai[opentelemetry]` which allows instrumentation via opentelemetry with LangChain in Azure AI Foundry.

## 2. Get the connection string to Application Insights

You can use the tracing capabilities in Azure AI Foundry by creating a tracer. Logs are stored in Azure Application Insights and can be queried at any time and hence you need a connection string to it. Each AI Hub has an Azure Application Insights created for you. You can get the connection string by **either**:

### Using the connection string directly:

In [ ]:
import os

application_insights_connection_string = os.environ["AZURE_APPINSIGHT_CONNECTION"]

### Using the Azure AI Foundry SDK

You can also get the connection string to Application Insights by using the Azure AI Foundry SDK along with the connection string to the project, as follows:

Install the Azure AI Foundry SDK:

```bash
pip install azure-ai-projects
```

In [ ]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str="<your-project-connection-string>",
)

application_insights_connection_string = project_client.telemetry.get_connection_string()

> You can find the project connection string in the landing page of your project.

## 3. Configure tracing for Azure AI Foundry

The following code creates a tracer connected to a project in Azure AI Foundry. Notice that the parameter `enable_content_recording` is set to `True`. This enables the capture of the inputs and outputs of the entire application as well as the intermediate steps. Such is helpful when debugging and building applications, but you may want to disable it on production environments. By default, the environment variable `AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED`:

In [ ]:
from langchain_azure_ai.callbacks.tracers import AzureAIInferenceTracer

tracer = AzureAIInferenceTracer(
    connection_string=application_insights_connection_string,
    enable_content_recording=True,
)

To see how tracers work, let's create a chain that uses multiple models and multiple steps. The following example generates a poem written by an urban poet:

In [ ]:
from langchain_core.prompts import PromptTemplate

producer_template = PromptTemplate(
    template="You are an urban poet, your job is to come up \
             verses based on a given topic.\n\
             Here is the topic you have been asked to generate a verse on:\n\
             {topic}",
    input_variables=["topic"],
)

verifier_template = PromptTemplate(
    template="You are a verifier of poems, you are tasked\
              to inspect the verses of poem. If they consist of violence and abusive language\
              report it. Your response should be only one word either True or False.\n \
              Here is the lyrics submitted to you:\n\
              {input}",
    input_variables=["input"],
)

We use two different models, one for the producer and one for the verifier. The producer has a more complicated task so we use a more powerful model:

In [ ]:
import os
from langchain_azure_ai.chat_models import AzureAIChatCompletionsModel

producer = AzureAIChatCompletionsModel(
    endpoint=os.environ["AZURE_INFERENCE_ENDPOINT"],
    credential=os.environ["AZURE_INFERENCE_CREDENTIAL"],
    model="mistral-large-2407",
)

verifier = AzureAIChatCompletionsModel(
    endpoint=os.environ["AZURE_INFERENCE_ENDPOINT"],
    credential=os.environ["AZURE_INFERENCE_CREDENTIAL"],
    model="mistral-small",
)

We can now combine the template, model, and the output parser from above using the pipe (`|`) operator:

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
generate_and_verify_chain = producer_template | producer | parser | verifier_template | verifier | parser

To configure tracing with your chain, indicate the value `config` in the `invoke` operation as a callback:

In [ ]:
generate_and_verify_chain.invoke({"topic": "living in a foreign country"}, config={"callbacks": [tracer]})

> Tip: the `generate_and_verify_chain` doesn't return the actual poem. This is done to facilitate the reading in this notebook as learning chains is not the main objective. In LangChain, to return also intermediate outputs from the chain you need to use `RunnableParallel` with `RunnablePassthrough()`.

To configure the chain itself for tracing, use the `.with_config()` method:

In [ ]:
generate_and_verify_chain_tracer = generate_and_verify_chain.with_config({"callbacks": [tracer]})

Then use the `invoke()` method as usual:

In [ ]:
generate_and_verify_chain_tracer.invoke({"topic": "living in a foreign country"})

Then, traces can be seen in Azure AI Foundry as follows:

![](docs/langchain-azure-ai/inference/tracing/portal-tracing-example.png)

![](../../docs/langchain-azure-ai/inference/tracing/portal-tracing-example.png)